#### Sprint 11 Project

In [1]:
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split

In [2]:
try:
    insurance_df = pd.read_csv('/datasets/insurance_us.csv')

except:
    insurance_df = pd.read_csv('insurance_us.csv')

#### EDA/ Data Preprocessing

In [3]:
insurance_df.head()

,Gender,Age,Salary,Family members,Insurance benefits
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


In [4]:
insurance_df = insurance_df.rename(columns={"Gender": "gender", "Age": "age", "Salary": "salary", "Family members": "family_members", "Insurance benefits": "insurance_benefits"})

In [5]:
insurance_df.isna().sum()

gender                0
age                   0
salary                0
family_members        0
insurance_benefits    0
dtype: int64

In [6]:
insurance_df.duplicated().sum()

153

In [7]:
insurance_df.drop_duplicates(inplace=True)

insurance_df.duplicated().sum()

0

In [8]:
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gender              4847 non-null   int64
 1   age                 4847 non-null   int64
 2   salary              4847 non-null   int64
 3   family_members      4847 non-null   int64
 4   insurance_benefits  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


#### Task 1:
Find customers who are similar to a given customer. This will help the company's agents with marketing.

documentation from Daniel:
https://scikit-learn.org/0.15/modules/generated/sklearn.neighbors.NearestNeighbors.html

scroll down and see examples of nearest neighbors algorithm in practice

no need for k neighbors classifier need nearest neighbors method in sklearn.
no need for target variable
given a row of data after fitting entire data, check what are n most similar rows.
can fit df into a nearest neighbors a 

In [40]:
t1_target = [insurance_df['gender'].loc[0], 
           insurance_df['age'].loc[0], 
           insurance_df['salary'].loc[0],
           insurance_df['family_members'].loc[0],
           insurance_df['insurance_benefits'].loc[0]]

t1_features = [insurance_df['gender'].loc[1:], 
           insurance_df['age'].loc[1:], 
           insurance_df['salary'].loc[1:],
           insurance_df['family_members'].loc[1:],
           insurance_df['insurance_benefits'].loc[1:]]

In [48]:
t1_model = KNeighborsClassifier(n_neighbors=2)

In [49]:
t1_model.fit(t1_features, t1_target)

KNeighborsClassifier(n_neighbors=2)

In [52]:
t1_model_predict

array([ 0,  1, 41,  1,  0], dtype=int64)

In [53]:
t1_model_predict = t1_model.predict(t1_features)

predict_df = pd.DataFrame(t1_model_predict, columns=['gender', 'age', 'salary', 'family_members', 'insurance_benefits'])

ValueError: Shape of passed values is (5, 1), indices imply (5, 5)

#### Task 2
Predict whether a new customer is likely to receive an insurance benefit. Can a trained prediction model do better than an untrained dummy model? Can it do worse? Explain your logic.

Need to read up on iterrows
class_balance_list = []

for index, row in insurance_df.iterrows():
    if row['insurance_benefits'] > 0:
        class_balance_list.append(1)
    else:
        class_balance_list.append(0)

insurance_df['class_balance'] = class_balance_list

In [12]:
insurance_df['recieved_benefits'] = (insurance_df['insurance_benefits'] > 0).astype(int)

In [13]:
t2_features = insurance_df.drop(['recieved_benefits'], axis=1)
t2_target = insurance_df['recieved_benefits']

t2_x_train, t2_x_test, t2_y_train, t2_y_test = train_test_split(t2_features, t2_target, test_size=0.3, random_state=12345)